In [1]:
#from my_snn.rsnn import RSNN, RSNN_2l, RSNN_2l_ALIF
from my_snn.rsnn_delays import RSNN_d_d, RSNN_d_i, RSNN_d_all
from my_snn.tonic_dataloader import DatasetLoader
from my_snn.utils import train, training_plots, ModelLoader
import torch
device = "cuda:0"

import matplotlib.pyplot as  plt

time_window = 250
batch_size = 256
dataset = 'shd'

DL = DatasetLoader(dataset=dataset, caching='disk', num_workers=0, batch_size=batch_size, time_window=time_window)
test_loader, train_loader = DL.get_dataloaders()

ckpt_dir = 'barridos_shd'

data = test_loader, train_loader

dropout = 0.2

import numpy as np
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader, dropout=dropout)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

D:\RESEARCHERS\Alberto\Anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_48_10_name = 'shd250_48x2_d_150_10_fs_8779296875'
model_64_10_name = 'shd250_64x2_d_150_10_fs_89013671875'
model_128_10_name = 'shd250_128x2_d_150_10_fs_89697265625'

In [3]:
for name in [model_128_10_name ]:
    for k in [1, 5, 10]:
        snn_pooled = ModelLoader(name, 'shd_soa_baselines', batch_size, 'cuda:0')
        snn_pooled.pool_delays('ho', k=k)
        snn_pooled.test(test_loader, dropout = dropout)
        snn_pooled.train_loss = []
        snn_pooled.test_loss = []
        snn_pooled.acc = []
        snn_pooled.modelname = f'{name}_pooled_{k}'
        train(snn_pooled, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
        torch.cuda.empty_cache()     

delays: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 2.9384197848183766
Avg spk_count per neuron for all 250 timesteps 5.810901641845703
Test Accuracy of the model on the test samples: 44.092
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=128, bias=False)
  (f1_f2): Linear(in_features=1920, out_features=128, bias=False)
  (f2_o): Linear(in_features=1920, out_features=20, bias=False)
)
Total params: 373760
Total mult-adds (M): 93.44
training shd250_128x2_d_150_10_fs_89697265625_pooled_1 for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 23.58575
Step [20/31], Loss: 21.25607
Step [30/31], Loss: 20.12482
Time elasped: 90.50633072853088
Epoch [2/50]
Step [10/31], Loss: 19.00226
Step [20/31], Loss: 18.34121
Step [30/31], Loss: 17.65106
Time elasped: 77.29947471618652
Epoch [3/50]
Step [10/31], Loss: 17.15594
Step [20/31], Loss: 16.40669
Step [30/31],